# Deep Learning Models (LOIO-CV Test)
- CNN: Convolutional Neural Network
- LSTM: Long Short-Term Memory
- DCL: DeepConvLSTM
- DCLSA: DeepConvLSTMSelfAttention 
- DCLSA-RN: ResNet version of DCLSA
- Transformer
- CNN-AE: CNN-based Auto-Encoder

# Import Modules

In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import torch

import sys
sys.path.append("../") # Set parent directory to sys.path
sys.dont_write_bytecode = True
%load_ext autoreload
%autoreload 2
from src import utils
from src.trainer import (
    test2, 
    generate_condition_list,
    generate_test_config_target,
    generate_results_save_path,
    load_and_setup_test_config,
    test_setup,
    generate_test_results_set,
    save_test_results_set
)

(
    plot_parameters, 
    okabe_ito_color_list, 
    tol_bright_color_list
) = utils.setup_plot(show_color_palette=False)
pd.set_option('display.width', 100)

confusion_matrix_parameters = {
    'font.size': 12,
    'axes.labelsize': 12,
    'legend.fontsize': 12,
    'figure.titlesize': 14,
    "figure.facecolor": "white"
}

In [ ]:
# check memory of nvidia cuda in dl-box
print(f"available devices: {torch.cuda.device_count()}")
!nvidia-smi

In [ ]:
TEST_CUDA_ID = 7

# Config

## issue

In [ ]:
issue = "I03"

## dataset

In [ ]:
# dataset = "om-50"
dataset = "um-50"

## ex

In [ ]:
# ex = "ex-d01" # flatten-linear or flatten-linear-2
# ex = "ex-d02" # dropout rate in flatten-linear-2

ex = "ex-d10" # data augmentation on DCL
# ex = "ex-d11" # data augmentation on DCL-SA

# ex = "ex-d15" # mixup argmax or not on DCL
# ex = "ex-d16" # mixup alpha w/ or w/o random data augmentation on DCL
# ex = "ex-d17" # mixup alpha after lstm layer of DCL

# ex = "ex-d20" # unsupervised pretraining of CNN-AE
# ex = "ex-d21" # no-freeze, soft-freeze, and hard-freeze using pretrained CNN-AE
# ex = "ex-d22" # CNN-AE w/o

# ex = "ex-d30" # model comparison

# ex = "ex-d60" # model hyperparameter tuning of DCLSA
# ex = "ex-d61" # model hyperparameter tuning of CNN-AE w/o

# ex = "ex-d70" # data augmentation hyperparameter tuning

In [ ]:
if ex in ["ex-d01", "ex-d02", "ex-d10", "ex-d15", "ex-d16"]:
    model_name = "dcl"
elif ex in ["ex-d11", "ex-d"]:
    model_name = "dcl-sa"
elif ex == "ex-d17":
    model_name = "dcl-v3"
elif ex == "ex-d21":
    model_name = "cnn-ae"
elif ex == "ex-d22":
    model_name = "cnn-ae-wo"
elif ex == "ex-d30":
    model_list = ['cnn', 'lstm', 'dcl', 'dcl-sa', 'resnet-l-sa', 'transformer', 'cnn-ae-wo']
    print(f"model_list: {model_list}")
    model_name = input("model_name: ")
    while model_name not in model_list:
        print(f"model_name: {model_name} is not in model_list")
        model_name = input("model_name: ")
elif ex == "ex-d60":
    model_name = "dcl-sa"
elif ex == "ex-d61":
    model_name = "cnn-ae-wo"
elif ex == "ex-d70":
    model_name = "dcl"

In [ ]:
print("--- Test config ---")
print(f"issue: {issue}")
print(f"ex: {ex}")
print(f"dataset: {dataset}")
print(f"model_name: {model_name}")

# Run Test

In [ ]:
checkpoints_fname = "best_model_weights.pt"

In [ ]:
condition_list = generate_condition_list(issue, ex, dataset, model_name)
utils.print_path_list_contents_with_index(condition_list)

In [ ]:
# seed_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
# seed_list = [0, 1, 2]
seed_list = [0]

In [ ]:
counter = 0
# check config target
for condition in condition_list:
    print(f"condition: {condition}")
    for seed in seed_list:
        (
            config_target,
            path_list
        ) = generate_test_config_target(issue, ex, dataset, model_name, condition, seed)
        print(f"config_target: {config_target}")
        config_path_list = sorted(glob.glob(config_target))
        print(f"N of target models: {len(config_path_list)}") # should be equal to the number of test loio-cv folds
        # utils.print_path_list_contents_with_index(config_path_list)
        
        for config_path in config_path_list:
            base_dir = os.path.dirname(config_path)
            best_model_path = f"{base_dir}/checkpoints_dir/best_model_weights.pt"
            if os.path.exists(best_model_path) == True:
                continue
            else:
                print("-----------------------")
                print(base_dir)
                print("-----------------------")
                counter += 1
print("----------------------------")
print(f"Error counter: {counter}")
print("----------------------------")

In [ ]:
# run test for each condition
for c, condition in enumerate(condition_list):
    print("-------------------------------------------------------------")
    print(f"condition: {condition}")
    
    # run test for each seed
    for s, seed in enumerate(seed_list):
        print(f"seed: {seed}")
        
        y_gt_all = []
        y_pred_all = []
        test_animal_id_all = []
        
        # prepare config path list
        config_target, path_list = generate_test_config_target(issue, ex, dataset, model_name, condition, seed)
        config_path_list = sorted(glob.glob(config_target))
        print(f"config_target path: {config_target}")
        print(f"N of target models: {len(config_path_list)}")
        
        # results save
        results_save_dir, f_basename = generate_results_save_path(path_list, checkpoints_fname)
        
        for i, config_path in enumerate(config_path_list):
            # load test config path
            cfg, DEVICE = load_and_setup_test_config(config_path, TEST_CUDA_ID, checkpoints_fname)
            
            # test setup 
            test_loader, best_model, optimizer, criterion = test_setup(cfg, config_path, DEVICE)

            # Run test
            (
                y_gt, y_pred, features, _cm, _df_cm, _fig_cm
            ) = test2(best_model, optimizer, criterion, test_loader, DEVICE, cfg)
            
            # array to list
            y_gt_list = y_gt.tolist()
            y_pred_list = y_pred.tolist()
            
            # store data for the final calculation
            y_gt_all.extend(y_gt_list)
            y_pred_all.extend(y_pred_list)
            test_animal_id_all.extend([cfg.dataset.test_animal_id]*len(y_gt_list))
        
        # Results set
        plt.rcParams.update(confusion_matrix_parameters)
        df_test_scores_all, df_gt_pred_all, fig_cm = generate_test_results_set(
            y_gt_all, 
            y_pred_all, 
            test_animal_id_all,
            cfg
        )
        
        # Save the results set
        save_test_results_set(results_save_dir, f_basename, df_test_scores_all, df_gt_pred_all)

        print("Saved!")
        
print("-------------------")
print("| Test completed. |")
print("-------------------")